In [15]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from keras.layers import TFSMLayer
from PIL import Image 
import cv2

# 1. 모델 불러오기
model = keras.models.load_model('./MobileNetV2_binary_modify.keras')

# 2. 테스트 이미지 파일 경로 설정
test_dir = './dataset/horse-or-human/test/'
  # 테스트 이미지가 있는 폴더 경로
image_size = (224, 224)  # 모델에 맞는 이미지 크기, 필요시 변경

/opt/anaconda3/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 322 variables whereas the saved optimizer has 326 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [16]:
# 3. 테스트 이미지 불러오기 및 전처리
def load_and_preprocess_image(img_path, target_size):
    try:
        # 이미지를 열고 강제로 RGB로 변환 (그레이스케일 이미지를 3차원으로 처리)
        img = Image.open(img_path).convert('RGB')
        
        # 이미지 크기 조정
        img = img.resize(target_size)
        
        # 이미지를 Numpy 배열로 변환 후 0-1 범위로 정규화
        img_array = np.array(img) / 255.0
        
        # 4차원 확장 (batch 차원 추가)
        img_array = np.expand_dims(img_array, axis=0)
        
        return img_array
    except Exception as e:
        print(f"Error loading image {img_path}: {e}")
        return None

# 4. 테스트 이미지와 레이블 불러오기
test_images = []
test_labels = []  # 실제 레이블 (0: horse, 1: human)
image_paths = [f for f in os.listdir(test_dir) if f.lower().endswith(('jpg', 'jpeg', 'png', 'webp'))]  # 이미지 파일만 선택
for img_path in image_paths:
    if 'horse' in img_path:  # 이미지 파일 이름에 'horse'이 포함된 경우
        test_labels.append(0)  # horse는 0
    elif 'human' in img_path:  # 이미지 파일 이름에 'human'이 포함된 경우
        test_labels.append(1)  # human은 1

    # 이미지 전처리
    img_array = load_and_preprocess_image(os.path.join(test_dir, img_path), image_size)
    if img_array is not None:
        test_images.append(img_array)

# Numpy 배열로 변환
if test_images:
    test_images = np.vstack(test_images)
    test_labels = np.array(test_labels)
else:
    print("No valid images were found.")

/opt/anaconda3/lib/python3.12/site-packages/PIL/Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [8]:
# 이미지 전처리 및 누락된 이미지 확인
test_images = []
test_labels = []  # 실제 레이블 (0: horse, 1: human)

for img_path in image_paths:
    if 'horse' in img_path:
        test_labels.append(0)  # horse는 0
    elif 'human' in img_path:
        test_labels.append(1)  # human은 1

    img_array = load_and_preprocess_image(os.path.join(test_dir, img_path), image_size)
    
    if img_array is not None and img_array.shape == (1, image_size[0], image_size[1], 3):  # 4차원 배열 확인
        test_images.append(img_array)
    else:
        print(f"Skipping image due to processing error: {img_path}")

print(f"Processed images: {len(test_images)}, Processed labels: {len(test_labels)}")


Processed images: 235, Processed labels: 235


In [14]:
# 5. 예측 수행
# 먼저 test_images를 Numpy 배열로 변환
test_images = np.vstack(test_images)  # 여기가 중요! 배치로 묶어야 함

# 이제 모델에 올바르게 예측을 요청할 수 있음
predictions = model.predict(test_images)  # 배치 입력으로 예측 수행
predicted_labels = np.argmax(predictions, axis=1)  # 클래스별 확률에서 가장 높은 클래스 선택

# 예측된 레이블과 실제 레이블의 개수 확인
print(f"Number of test labels: {len(test_labels)}")
print(f"Number of predicted labels: {len(predicted_labels)}")

# 레이블 개수가 일치하지 않을 때 경고 메시지 출력
if len(test_labels) != len(predicted_labels):
    print("Mismatch in label counts!")
    print(f"Test labels: {len(test_labels)}, Predicted labels: {len(predicted_labels)}")
else:
    # 6. 성능 지표 계산 (개수가 일치할 때만 계산)
    accuracy = accuracy_score(test_labels, predicted_labels)
    precision = precision_score(test_labels, predicted_labels)
    recall = recall_score(test_labels, predicted_labels)
    f1 = f1_score(test_labels, predicted_labels)
    conf_matrix = confusion_matrix(test_labels, predicted_labels)

    # 성능 지표 출력
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-Score: {f1}")
    print(f"Confusion Matrix:\n{conf_matrix}")


8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 655ms/step
Number of test labels: 235
Number of predicted labels: 235
Accuracy: 0.5191489361702127
Precision: 0.0
Recall: 0.0
F1-Score: 0.0
Confusion Matrix:
[[122   0]
 [113   0]]


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
